# Import

In [1]:
!pip install -q -U bitsandbytes --no-index --find-links /kaggle/input/pip-llm-0729
!pip install -q -U transformers --no-index --find-links /kaggle/input/pip-llm-0729
!pip install -q -U tokenizers --no-index --find-links /kaggle/input/pip-llm-0729
!pip install -q -U peft --no-index --find-links /kaggle/input/pip-llm-0729

In [2]:
import time
import torch
import sklearn
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.cuda.amp import autocast
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from threading import Thread
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType 
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import BitsAndBytesConfig, LlamaForCausalLM, LlamaModel, LlamaPreTrainedModel
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from transformers import set_seed
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)
assert torch.cuda.device_count() == 2, "Sorry - multi-GPU required!"

2024-08-04 22:00:05.691035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 22:00:05.691189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 22:00:05.844660: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
MODEL_NAME = '/kaggle/input/llama3-1-it'
WEIGHTS_PATH = '/kaggle/input/llama3-1-8b-base2-final'    # 自作モデルの重みパス
MAX_LENGTH = 1800     # 最大トークン長
BATCH_SIZE = 2        # 予測出力の際のバッチサイズ
DEVICE = torch.device("cuda")      # GPUを利用

# Prepare Data 

 - label_token_idのコンフィグデータ

    "128250": {  
          "content": "<|reserved_special_token_242|>",  
          "lstrip": false,  
          "normalized": false,  
          "rstrip": false,  
          "single_word": false,  
          "special": true  
        },  

In [ ]:
# モデルへの入力データを成形（文章内のnull文字列は無視する）

test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
def tokenize(example, tokenizer):
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    
    # 最大長を超える際に各文章の上限を設定（動的なトークン長調整と精度差なし）
    if len(prompt+response_a+response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:350]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:725]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:725]
    # 生成AIとしてextra_promptの後の単語を出力
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]
    # モデルconfigファイルのスペシャルトークンid
    label_token_id = [128250]
    # 入力データの連結
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + label_token_id + [tokenizer.eos_token_id]
    attention_mask = len(input_ids)*[1]
    labels = [-100]* len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + label_token_id + [tokenizer.eos_token_id]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# Tokenize

In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)    # Llama3.1デフォルトのトークナイザー
tokenizer.add_eos_token = True      # 文章の終点idを有効化
tokenizer.pad_token = tokenizer.eos_token     # padding_idとeos_idを揃える
tokenizer.padding_side = 'right'     # 右からパディングする

LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]
def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        # remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer},
    )
    return tokenized_datasets
test_ds = load_data(test, tokenizer)
test_ds

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

CPU times: user 630 ms, sys: 66.6 ms, total: 696 ms
Wall time: 801 ms


Dataset({
    features: ['id', 'prompt', 'response_a', 'response_b', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3
})

# Confirmation

In [6]:
data = test_ds.to_pandas()
data["max_len"] = data["input_ids"].apply(len)
data[:3]

,id,prompt,response_a,response_b,input_ids,attention_mask,labels,max_len
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran...","[128000, 8085, 15091, 27916, 358, 617, 2380, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",82
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ...","[128000, 8085, 15091, 27916, 1472, 527, 264, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",469
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p...","[128000, 8085, 15091, 27916, 2650, 311, 9656, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",1593


In [7]:
data['input_ids'][0]

array([128000,   8085,  15091,  27916,    358,    617,   2380,  85138,
         3432,     11,    358,  30912,    459,  19087,  13985,     13,
         2650,   1690,  85138,    656,    358,    617,     30,    271,
           27,   2376,   4404,  27916,   1472,    617,   1403,  85138,
         3432,     13,    271,     27,   2376,    890,  27916,   1472,
         2103,    617,   2380,  85138,     13,  60638,    459,  19087,
        13985,   1587,    539,   7958,    279,   1396,    315,  85138,
          499,    617,   3432,     13,    271,  29547,  23956,    374,
          279,   2731,   2077,    369,    279,  10137,    949,    264,
          477,    293,    477,  18623,  24688,  16533,     25,    220,
       128250, 128009], dtype=int32)

In [8]:
print(tokenizer.decode(data["input_ids"][0]))

<|begin_of_text|><prompt>: I have three oranges today, I ate an orange yesterday. How many oranges do I have?

<response_a>: You have two oranges today.

<response_b>: You still have three oranges. Eating an orange yesterday does not affect the number of oranges you have today.

---------
Which is the better response for the prompt? a or b or tie?

Answer: <|reserved_special_token_242|><|eot_id|>


# Load model 
We load 1 model on each gpu.  

In [9]:
class Llama3ForSFT(LlamaPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.post_init()

    def forward(
        self,
        input_ids= None,
        attention_mask= None,
        position_ids = None,
        past_key_values= None,
        inputs_embeds= None,
        labels= None,
        use_cache= None,
        output_attentions= None,
        output_hidden_states = None,
        return_dict= None,
        cache_position = None,
    ):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]
        if self.config.pretraining_tp > 1:
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)
        else:
            logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)

            fake_label_tokens_ids = torch.tensor([128250],device=shift_labels.device)
            label_tokens_ids = torch.tensor(LABEL_IDS,device=shift_labels.device)
#             index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
#             true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]
#             true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)
            true_logits = shift_logits[torch.isin(shift_labels, fake_label_tokens_ids)][:,label_tokens_ids]
#             loss = loss_fct(true_logits, true_labels)

        return CausalLMOutputWithPast(
            loss=loss,
            logits=true_logits,
        )

In [ ]:
bnb_config =  BitsAndBytesConfig(
        load_in_8bit=True,
    )
# Load base model on GPU 0
device0 = torch.device('cuda:0')
base_model_0 = Llama3ForSFT.from_pretrained(
    MODEL_NAME,
    use_cache=False,
    quantization_config=bnb_config,
    device_map='cuda:0',
)
# Load base model on GPU 1
device1 = torch.device('cuda:1')
base_model_1 = Llama3ForSFT.from_pretrained(
    MODEL_NAME,
    use_cache=False,
    quantization_config=bnb_config,
    device_map='cuda:1',
)

# base_model_0.config.pad_token_id = tokenizer.pad_token_id
# base_model_1.config.pad_token_id = tokenizer.pad_token_id


# Load weights 

In [11]:
# Get peft
model_0 = PeftModel.from_pretrained(base_model_0, model_id=WEIGHTS_PATH).to(device0) 
model_0.eval()

model_1 = PeftModel.from_pretrained(base_model_1, model_id=WEIGHTS_PATH).to(device1)
model_1.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Llama3ForSFT(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

# Inference


In [12]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=BATCH_SIZE, max_length=MAX_LENGTH):
    a_win, b_win, tie = [], [], []

    model.eval()
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        labels = tmp["labels"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        pad_labels=[]
        for label in labels:
            label = list(label) + [tokenizer.pad_token_id]*(input_ids[0].shape[0]-label.shape[0])
            pad_labels.append(label)
        labels = torch.tensor(pad_labels).to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        proba = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    df['winner_model_a'] = a_win
    df['winner_model_b'] = b_win
    df['winner_tie'] = tie
    return df

In [13]:
st = time.time()

data = data.sort_values("max_len", ascending=False)
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device0, device1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")

elapsed time: 3.601780891418457


In [14]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)

,id,winner_model_a,winner_model_b,winner_tie
2,1233961,0.106284,0.657455,0.236261
0,136060,0.016231,0.910743,0.073027
1,211333,0.360373,0.280659,0.358968
